# Analysis of CIA factbook data

This project deals with analysis of the [CIA factbook data](https://www.cia.gov/the-world-factbook/). The CIA factbook contains demographic information like the country's area, population, population growth, birth rate and so on.

## Connecting to the database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

 ## Overview of Data
 Let's see how the first 5 rows of data looks like

In [2]:
%%sql
SELECT * 
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Description of Columns
* <b>code</b> - country code
* <b>name</b> - name of the country
* <b>area</b>- total area of the country both land and water in square kilometers.
* <b>area_land</b>- area occupied by the country in land in square kilometers.
* <b>area_water</b>- area occupied by the country in water in square kilometers.
* <b>population</b>- total population of the country
* <b>population_growth</b>- population growth rate of the country as percentages
* <b>birth_rate </b>- number of birth per year per 1000 people
* <b>death_rate </b>- number of deaths per year per 1000 people
* <b>migration_rate </b>- change in population due to migration per 1000 people

## Summary Statistics
Let's calculate the maximum and minimum of Population, population growth

In [3]:
%%sql
SELECT  MAX(population) AS maximum_population,
        MIN(population) AS minimum_population,
        MAX(population_growth) AS maximum_population_growth,
        MIN(population_growth) AS minimum_population_growth  
    FROM facts;

Done.


maximum_population,minimum_population,maximum_population_growth,minimum_population_growth
7256490011,0,4.02,0.0


### Interesting information in the data:

* A country with 7.25 billion inhabitants.
* A country with 0 inhabitants.

Let's write a query to check the name of the country with maximum population

In [4]:
%%sql 
SELECT name AS country, MAX(population) AS max_population
    FROM facts;

Done.


country,max_population
World,7256490011


The data consists of a row with the data of the World and that explains why there is 7.25 billion as maximum population.
Let's write a query to figure out the country with 0 inhabitants.

In [5]:
%%sql
SELECT name AS country, MIN(population) AS minimum_population
    FROM facts;

Done.


country,minimum_population
Antarctica,0


The data contains Antarctica which explains the minimum number of population as 0. The image here shows the a screenshot taken from the CIA factbook website and also says that there are no indegeneous inhabitants. The relevant information can be found in people and society section of [CIA FACTBOOK Antarctica section](https://www.cia.gov/the-world-factbook/countries/antarctica/).
<img src="antarctica.png"/>

## Summary statistics excluding the World
Let's calculate the summary statistics without the world row.

In [6]:
%%sql
SELECT  MIN(population) AS minimum_population,
        MAX(population) AS maximum_population,
        MAX(population_growth) AS maximum_population_growth,
        MIN(population_growth) AS minimum_population_growth
    FROM facts
    WHERE name <> "World";

Done.


minimum_population,maximum_population,maximum_population_growth,minimum_population_growth
0,1367485388,4.02,0.0


The maximum population turn outs to be 1.36 billion. Let's look in the country which this maximum population belongs to.

In [7]:
%%sql
SELECT  name AS country, 
        MAX(population) AS maximum_population
    FROM facts
    WHERE country <> "World";

Done.


country,maximum_population
China,1367485388


So, <b>China</b> is the country with maximum population.

## AVERAGE AREA
Let's  calculate the average area excluding the world

In [8]:
%%sql
SELECT ROUND(AVG(area),2) AS average_area
    FROM facts 
    Where name <> "World";

Done.


average_area
555093.55


The data reveals that the average area is a bit greater than five hundred and fifty five thousand.
Now, let's extract the list of the countries which has area greater than average area.

In [9]:
%%sql
SELECT * 
    FROM facts
    WHERE area > (SELECT AVG(area) 
                     FROM facts
                     WHERE name <> "World")
    AND
    name <> "World"
    ORDER BY area DESC;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
87,kz,Kazakhstan,2724900,2699700,25200,18157122,1.14,19.15,8.21,0.41


<b>Russia</b> is at the top of the list and <b>Kenya</b> is at the bottom. Let's count the number of countries which has area greater than average area.

In [10]:
%%sql 
SELECT COUNT(*) greater_than_average_count 
    FROM facts
    WHERE area > (SELECT AVG(area) 
                     FROM facts
                     WHERE name <> "World")
    AND
    name <> "World";

Done.


greater_than_average_count
48


There are 48 countries which has area greater than the average area.

## Countries with Water more than Land

In [11]:
%%sql
SELECT name AS country, ROUND(CAST(area_water AS FLOAT)/area_land, 5) AS water_land_ratio
    FROM facts
    WHERE name <> "World"
    AND
    water_land_ratio > 1
    ORDER BY water_land_ratio DESC;

Done.


country,water_land_ratio
British Indian Ocean Territory,905.66667
Virgin Islands,4.52023


The data shows <b>British Indian Ocean Territory</b> and <b>Virgin Islands</b> have more water area than land.

## Which country will add most people to their population next year ? 

In [12]:
%%sql
SELECT  name AS country,
        MAX((population_growth/100) * population) AS highest_population_increase
    FROM facts
    where country <> "World";

Done.


country,highest_population_increase
India,15270686.124799998


<b> India </b> will add the most population next year which is around 15.27 million.

## List of countries with Higher death rate than birth rate

In [13]:
%%sql
SELECT  name AS country, birth_rate, death_rate
    FROM facts
    WHERE death_rate > birth_rate
    AND
    country <> "World"
    ORDER BY death_rate DESC;

Done.


country,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


In [14]:
%%sql
SELECT  COUNT(*) AS country_count
    FROM facts
    WHERE death_rate > birth_rate
    AND
    name <> "World";

Done.


country_count
24


* There are 24 countries with death rate higher than birth rate.
* <b> Ukraine </b> is at the top of the list with rate of <b>14.46</b> per <b>1000</b> people.
* <b> Monaco </b> is at the bottom of the list with rate of <b>9.24</b> per <b>1000</b> people.

## Population Density

The total number of people per unit area defines the population density. Let's take a look at list of countries in accordance with population density

In [15]:
%%sql
SELECT  name AS country,
        ROUND(CAST(population AS FLOAT)/area, 3) AS population_density
    FROM facts
    WHERE country <> "World"
    ORDER BY population_density DESC
    LIMIT 10;

Done.


country,population_density
Macau,21168.964
Monaco,15267.5
Singapore,8141.28
Hong Kong,6445.042
Gaza Strip,5191.819
Gibraltar,4876.333
Bahrain,1771.859
Maldives,1319.641
Malta,1310.016
Bermuda,1299.926


<b>Macau</b> is the one with highest population density.